In [1]:
import pandas as pd
import shapefile
import shapely

In [2]:
polygons_sf = shapefile.Reader("/Users/damoncrockett/Desktop/FISP/SanGIS/Parcels/PARCELS.shp")
polygon_shapes = polygons_sf.shapes()
polygon_points = [q.points for q in polygon_shapes]
from shapely.geometry import Polygon, Point
polygons = [Polygon(q) for q in polygon_points]

In [3]:
records = polygons_sf.records()

In [5]:
fields = polygons_sf.fields

In [6]:
len(fields)

60

In [12]:
fields[:5]

[('DeletionFlag', 'C', 1, 0),
 ['APN', 'C', 10, 0],
 ['APN_8', 'C', 8, 0],
 ['PARCELID', 'N', 10, 0],
 ['OWN_NAME1', 'C', 96, 0]]

In [8]:
len(records[0])

59

In [11]:
records[0][:5]

['2731740600',
 '27317406',
 387979,
 'CALDWELL TRUST 09-22-03',
 '                                                  ']

Ok so although the 'fields' output has this 'deletion flag' thing, it doesn't actually appear in the records.

In [18]:
cities = []
for record in records:
    cities.append(record[12])

In [19]:
from collections import Counter

In [20]:
Counter(cities)

Counter({'SD': 382766, 'CN': 218433, 'CB': 89832, 'CV': 66421, 'OC': 61000, 'ES': 37297, 'SM': 25856, 'VS': 23848, 'EN': 23121, 'EC': 22577, 'ST': 18446, 'LM': 17194, 'PW': 16111, 'SO': 13097, 'CO': 11042, 'NC': 10287, 'LG': 7253, 'IB': 5959, 'DM': 4669, '  ': 1255})

In [21]:
sd = []
for record in records:
    if record[12]=='SD':
        sd.append(record)

In [22]:
len(sd)

382766

In [34]:
columns = fields[1:]

In [35]:
cols = []
for column in columns:
    cols.append(column[0])

In [40]:
len(sd[0])

59

In [45]:
df = pd.DataFrame()

In [46]:
for i in range(59):
    tmp = []
    for record in sd:
        tmp.append(record[i])
    df[i] = tmp

In [49]:
df.columns = cols

In [55]:
df.iloc[:,:7].head()

,APN,APN_8,PARCELID,OWN_NAME1,OWN_NAME2,OWN_NAME3,FRACTINT
0,3732103400,37321034,479921,MARTORANA ELLEN,...,...,1
1,3181344500,31813445,467579,JESTER EDGAR L JR,...,...,1
2,5830411300,58304113,115047,ARCHER MARIA J,...,...,1
3,3532400900,35324009,5122202,WINDEMERE HOMEOWNERS ASSN,...,...,1
4,5390213300,53902133,255959,ZIMMER PAUL&KATHERINE C,...,...,1


In [57]:
n = len(df)

In [66]:
l = list(df.columns)
l = [item.lower() for item in l]

In [68]:
df.columns = l

In [70]:
addresses = []
for i in range(n):
    if df.situs_addr.loc[i]!=None:
        a = str(df.situs_addr.loc[i])
    else:
        a = ''
    if df.situs_pre_.loc[i]!=None:
        b = df.situs_pre_.loc[i]
    else:
        b = ''
    if df.situs_stre.loc[i]!=None:
        c = df.situs_stre.loc[i]
    else:
        c = ''
    if df.situs_suff.loc[i]!=None:
        d = df.situs_suff.loc[i]
    else:
        d = ''
    if df.situs_post.loc[i]!=None:
        e = df.situs_post.loc[i]
    else:
        e = ''
    if df.situs_frac.loc[i]!=None:
        f = df.situs_frac.loc[i]
    else:
        f = ''
    g = 'SAN DIEGO'
    if df.situs_zip.loc[i]!=None:
        h = str(df.situs_zip.loc[i])
    else:
        h = ''
    tmp = a+' '+b+' '+c+' '+d+' '+e+' '+f+' '+g+' '+h
    addresses.append(tmp)

In [72]:
df['address'] = addresses

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382766 entries, 0 to 382765
Data columns (total 60 columns):
apn           382766 non-null object
apn_8         382766 non-null object
parcelid      382766 non-null int64
own_name1     382766 non-null object
own_name2     382766 non-null object
own_name3     382766 non-null object
fractint      382766 non-null float64
own_addr1     382766 non-null object
own_addr2     382766 non-null object
own_addr3     382766 non-null object
own_addr4     382766 non-null object
own_zip       382766 non-null object
situs_juri    382766 non-null object
situs_stre    382766 non-null object
situs_suff    382766 non-null object
situs_post    382766 non-null object
situs_pre_    382766 non-null object
situs_addr    382766 non-null int64
situs_frac    382766 non-null object
situs_buil    382766 non-null object
situs_suit    382766 non-null object
situs_comm    382766 non-null object
legldesc      382766 non-null object
asr_land      382766 non-null int64
asr

In [74]:
df.to_csv("/Users/damoncrockett/Desktop/FISP/parcels_new.csv",index=False)